## Ejercicio/Tarea Abraham Nieto 51556

Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador `delayed` para ejecutar en paralelo.

* ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Haz lo mismo que arriba, pero utilizando la biblioteca Dask-ML http://dask-ml.readthedocs.io/en/latest/ 

* ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

**Bonus**

Haz lo mismo utilizando Spark ML

* ¿Cómo se comparan los tiempos de ejecución de Spark vs Dask?

Usa los datos en s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv

* ¿Cambia alguno de los resultados anteriores?

In [1]:
from dask import dataframe
trips_df = dataframe.read_csv("/data/trips.csv")
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00


In [2]:
# Truco para convertir a datetime...
trips_df.tpep_pickup_datetime = trips_df.tpep_pickup_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime = trips_df.tpep_dropoff_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime.head()

0   2015-01-03 01:37:02
1   2015-01-05 23:35:02
2   2015-01-06 15:22:12
3   2015-01-08 08:31:23
4   2015-01-08 12:35:54
Name: tpep_dropoff_datetime, dtype: datetime64[ns]

In [3]:
# Agrega una columna para la duración del viaje
trips_df["duracion"]=trips_df["tpep_dropoff_datetime"]-trips_df["tpep_pickup_datetime"]
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,duracion
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,00:19:30
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,00:09:47
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,00:10:27
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,00:09:11
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,00:09:28


In [4]:
# ¿Cuál es la proporción de propina promedio para los viajes con más de 3 pasajeros?
trips_df["propor"]=trips_df["tip_amount"]/trips_df["fare_amount"] 
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,duracion,propor
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,00:19:30,0.209091
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,00:09:47,0.000000
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,00:10:27,0.133333
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,00:09:11,0.117647
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,00:09:28,0.221333


In [5]:
import numpy as np 
import time
from datetime import datetime, timedelta
from sklearn.cross_validation import train_test_split 

trips_var=trips_df[["car_type","passenger_count","trip_distance","duracion","propor"]]
trips_var.duracion=trips_var.duracion/ timedelta (days=1) 
trips_var=trips_var.fillna(trips_var.mean()).compute()
trips_var.head()

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,car_type,passenger_count,trip_distance,duracion,propor
0,A,1,6.90,0.013542,0.209091
1,A,1,1.81,0.006794,0.000000
2,A,1,0.96,0.007257,0.133333
3,A,1,1.90,0.006377,0.117647
4,A,1,1.00,0.006574,0.221333


In [6]:
X=np.array(trips_var[["car_type","passenger_count","trip_distance","duracion"]])
y=np.array(trips_var['propor']) 
#data partition
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=6)


In [1]:
from sklearn.preprocessing import StandardScaler,LabelEncoder,FunctionTransformer,OneHotEncoder
from sklearn.pipeline import make_pipeline 
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
class ColumnExtractor(object):

    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

In [2]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                #print(col)
                output[:,col] = LabelEncoder().fit_transform(output[:,col])
        #else:
        #    for colname,col in output.iteritems():
         #       output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [9]:
#Hacemos diccionario de variables dado que la función FCB trabaja con dataframes necesitaremos un array 
#de salida y así ligar las variables a la columna correspondiente del array.
#"car_type","passenger_count","trip_distance","duracion"
my_dict = {"car_type" : 0,  "passenger_count": 1,"trip_distance" : 2, "duracion" : 3}

#Hacemos unas funciones que distinga las variables numéricas y categóricas en un numpy.matrix a partir 
#del dataframe. 

def vartipo(df):
    nm=df.select_dtypes(['int32','float32','float64','int64','timedelta64']) 
    num=list(nm.columns)
    cat=df.select_dtypes(['object']) 
    nom=list(cat.columns)
    return(num,nom)
def coltipo(d,df):
    n=[]
    c=[]
    num,nom=vartipo(df)
    num.remove('propor') 
    for nm in num:
        n.append(d[nm])
    for ct in nom:
        c.append(d[ct])
       
    return(n,c,num,nom)    



In [10]:
#trips_var.info()

In [36]:
#Hacemos un pequeño pipeline que separa las variables en numéricas y categóricas y hace escalamiento y codifica
#las categóricas y luego las junta.
n,c,num,nom=coltipo(my_dict,trips_var)
pip=Pipeline([('features', FeatureUnion([
        ('continuous',Pipeline([("nums", ColumnExtractor(cols=(n))), ("sc",StandardScaler())])),
        ('factors',Pipeline([("cat", ColumnExtractor(cols=(c))), 
                             ("oe",MultiColumnLabelEncoder(columns=(list(range(len(c))))))])
)]))])

In [42]:
ax=pip.fit_transform(X_train).astype('float64')

/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [43]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LinearRegression  
regressor = LinearRegression()  
regressor.fit(ax, y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

**Secuencial**

In [44]:
%%time
from dask import delayed
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(regressor,parameters, cv=10)
grid.fit(ax, y_train)
print("r2 / variance : ", grid.best_score_)
#print("Residual sum of squares: %.2f"
#              % np.mean((grid.predict(X_test) - y_test) ** 2))

r2 / variance :  -0.0013954063593684804
CPU times: user 129 ms, sys: 897 µs, total: 130 ms
Wall time: 127 ms


**Paralelo**

In [45]:
%%time
from dask import delayed
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = delayed(GridSearchCV(regressor,parameters, cv=10))
grid.fit(ax, y_train)
print("r2 / variance : ", grid.best_score_)

r2 / variance :  Delayed('getattr-24bcc0ba76df7461c988cdb3d718cd55')
CPU times: user 1.2 ms, sys: 0 ns, total: 1.2 ms
Wall time: 714 µs


**¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?**


In [46]:
print("es ",(1-(7.14/127))*100,"% mas rapido")

es  94.37795275590551 % mas rapido


**¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?**

In [47]:
#from dask_ml.linear_model import LinearRegression
from dask_searchcv import GridSearchCV, RandomizedSearchCV

In [48]:
%%time
clf = LinearRegression()

# run grid search
dk_grid_search = GridSearchCV(clf, param_grid=parameters, n_jobs=-1,cv=10)
dk_grid_search.fit(ax, y_train)

CPU times: user 319 ms, sys: 1.07 s, total: 1.39 s
Wall time: 195 ms


es mas lento con daskml  195ms vs 714$mu$s

In [49]:
print("r2 / variance : ", dk_grid_search.best_score_)

r2 / variance :  -0.0013954063593684804


**Bonus**

In [3]:
import pandas as pd
import dask.dataframe as dd
from distributed import Client, progress
from dask import persist, compute
#from dask_ml.linear_model import LogisticRegression

#client = Client("scheduler:8786")
ddf = dd.read_csv('/data/yellow_tripdata_2015-01.csv')

In [4]:
ddf.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


In [5]:
import numpy as np
ddf["propor"]=ddf["tip_amount"]/ddf["fare_amount"] 
#ddf=ddf.fillna(0).compute()
#ddf=ddf[~np.isinf(ddf["propor"])]
ddf.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,propor
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05,0.270833
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80,0.137931
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80,0.000000
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.000000
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30,0.000000


In [6]:
d_var=ddf[["trip_distance","store_and_fwd_flag","payment_type","extra","mta_tax",
           "tolls_amount","improvement_surcharge","propor"]]
d_var=d_var.fillna(0).compute()


In [7]:
d_var=d_var[~np.isinf(d_var["propor"])]

In [8]:
from sklearn.cross_validation import train_test_split 
import numpy as np 
X=np.array(d_var[["trip_distance","store_and_fwd_flag","payment_type","extra","mta_tax","tolls_amount","improvement_surcharge"]])
y=np.array(d_var['propor']) 
#data partition
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=6)

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
#Hacemos diccionario de variables dado que la función FCB trabaja con dataframes necesitaremos un array 
#de salida y así ligar las variables a la columna correspondiente del array.
#"car_type","passenger_count","trip_distance","duracion"
my_dict = {"trip_distance" : 0,  "store_and_fwd_flag": 1,"payment_type" : 2, "extra" : 3,
          "mta_tax":4,"tolls_amount":5,"improvement_surcharge":6}

#Hacemos unas funciones que distinga las variables numéricas y categóricas en un numpy.matrix a partir 
#del dataframe. 

def vartipo(df):
    nm=df.select_dtypes(['int32','float32','float64','int64','timedelta64']) 
    num=list(nm.columns)
    cat=df.select_dtypes(['object']) 
    nom=list(cat.columns)
    return(num,nom)
def coltipo(d,df):
    n=[]
    c=[]
    num,nom=vartipo(df)
    num.remove('propor') 
    for nm in num:
        n.append(d[nm])
    for ct in nom:
        c.append(d[ct])
       
    return(n,c,num,nom)   

In [10]:
#Hacemos un pequeño pipeline que separa las variables en numéricas y categóricas y hace escalamiento y codifica
#las categóricas y luego las junta.
n,c,num,nom=coltipo(my_dict,d_var)
pip=Pipeline([('features', FeatureUnion([
        ('continuous',Pipeline([("nums", ColumnExtractor(cols=(n))), ("sc",StandardScaler())])),
        ('factors',Pipeline([("cat", ColumnExtractor(cols=(c))), 
                             ("oe",MultiColumnLabelEncoder(columns=(list(range(len(c))))))])
)]))])

In [11]:
ax=pip.fit_transform(X_train).astype('float64')

/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
ax

array([[-1.09709103e-03, -7.75250238e-01,  3.59689005e-01, ...,
        -1.62897608e-01,  2.43939544e-01,  0.00000000e+00],
       [-1.39389034e-03, -7.75250238e-01,  3.59689005e-01, ...,
        -1.62897608e-01,  2.43939544e-01,  0.00000000e+00],
       [-1.39389034e-03, -7.75250238e-01, -5.78219520e-01, ...,
        -1.62897608e-01,  2.43939544e-01,  0.00000000e+00],
       ...,
       [-1.43641173e-03,  1.22951420e+00,  3.59689005e-01, ...,
        -1.62897608e-01,  2.43939544e-01,  0.00000000e+00],
       [-1.43641173e-03, -7.75250238e-01,  3.59689005e-01, ...,
        -1.62897608e-01,  2.43939544e-01,  0.00000000e+00],
       [-1.26752883e-04,  1.22951420e+00, -5.78219520e-01, ...,
        -1.62897608e-01,  2.43939544e-01,  0.00000000e+00]])

In [13]:
%%time
from sklearn.linear_model import LinearRegression  
from dask_searchcv import GridSearchCV, RandomizedSearchCV
clf = LinearRegression()
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}

# run grid search
dk_grid_search = GridSearchCV(clf, param_grid=parameters, n_jobs=-1,cv=10)
dk_grid_search.fit(ax, y_train)

CPU times: user 5min 12s, sys: 5min 26s, total: 10min 38s
Wall time: 6min 26s


In [14]:
print("r2 / variance : ", dk_grid_search.best_score_)

r2 / variance :  -0.0007188674738379506


In [15]:
np.shape(ax)

(8924094, 7)

**pyspark no termina el crossvalidation pero se deja codigo para ejecutar**

In [12]:
import pyspark
import os
import sys
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql import SparkSession 
sc = SparkContext()
sqlContext = SQLContext(sc) 

In [13]:
np.shape(ax)
yax=y_train.reshape(8924094,1)

In [ ]:
from pyspark.ml.linalg import Vectors
df=np.concatenate([yax,ax],axis=1)
dff = map(lambda x: (int(x[0]), Vectors.dense(x[1:])), df)
mydf = sqlContext.createDataFrame(dff,schema=["label", "features"])

In [31]:
train_data,test_data = mydf.randomSplit([0.1,0.9])

In [32]:
from pyspark.ml.regression import LinearRegression 
lr = LinearRegression(featuresCol="features",labelCol='label')

In [33]:
# parameter grid
#fitIntercept=True, standardization=True
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(lr.fitIntercept, [True,False]).\
    addGrid(lr.standardization, [True,False]).\
    build()

In [36]:
from pyspark.ml.evaluation import RegressionEvaluator 
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=param_grid,
                          evaluator=RegressionEvaluator(predictionCol='prediction', labelCol="label", 
                               metricName='r2'), #default es rmse
                          numFolds=10)

**no termina el cross validation**

In [12]:
#%%time 
#cvModel = crossval.fit(train_data)

**¿Cómo se comparan los tiempos de ejecución de Spark vs Dask?**

6min 26s vs.  10% de ax para probar que pyspark es mas lento el proceso de pyspark no termino...

**¿Cambia alguno de los resultados anteriores?**